# Patch daily with jura

In [1]:
import os
import numpy as np
from astropy.table import Table, join
from astropy.io import fits
from desispec.io import read_table
from desispec.efftime import compute_efftime
from specprodDB.util import cameraid

In [2]:
daily_tiles_file = os.path.join(os.environ['DESI_SPECTRO_REDUX'], 'daily', 'tiles-daily.csv')
daily_exposures_file = os.path.join(os.environ['DESI_SPECTRO_REDUX'], 'daily', 'exposures-daily.fits')
jura_tiles_file = os.path.join(os.environ['DESI_SPECTRO_REDUX'], 'jura', 'tiles-jura.csv')
jura_exposures_file = os.path.join(os.environ['DESI_SPECTRO_REDUX'], 'jura', 'exposures-jura.fits')
# iron_tiles_file = os.path.join(os.environ['DESI_SPECTRO_REDUX'], 'iron', 'tiles-iron.csv')
# iron_exposures_file = os.path.join(os.environ['DESI_SPECTRO_REDUX'], 'iron', 'exposures-iron.fits')
# fuji_tiles_file = os.path.join(os.environ['DESI_SPECTRO_REDUX'], 'fuji', 'tiles-fuji.csv')
# fuji_exposures_file = os.path.join(os.environ['DESI_SPECTRO_REDUX'], 'fuji', 'exposures-fuji.fits')
daily_tiles = Table.read(daily_tiles_file, format='ascii.csv')
daily_exposures = Table.read(daily_exposures_file, format='fits', hdu='EXPOSURES')
# daily_exposures = read_table(daily_exposures_file, ext='EXPOSURES')
daily_frames = Table.read(daily_exposures_file, format='fits', hdu='FRAMES')
# daily_frames = read_table(daily_exposures_file, ext='FRAMES')
jura_tiles = Table.read(jura_tiles_file, format='ascii.csv')
jura_exposures = Table.read(jura_exposures_file, format='fits', hdu='EXPOSURES')
# jura_exposures = read_table(jura_exposures_file, ext='EXPOSURES')
jura_frames = Table.read(jura_exposures_file, format='fits', hdu='FRAMES')
# jura_frames = read_table(jura_exposures_file, ext='FRAMES')
# iron_tiles = Table.read(iron_tiles_file, format='ascii.csv')
# iron_exposures = read_table(iron_exposures_file, ext='EXPOSURES')
# iron_frames = read_table(iron_exposures_file, ext='FRAMES')
# fuji_tiles = Table.read(fuji_tiles_file, format='ascii.csv')
# fuji_exposures = read_table(fuji_exposures_file, ext='EXPOSURES')
# fuji_frames = read_table(fuji_exposures_file, ext='FRAMES')

## Do some QA on jura

In [ ]:
for c in jura_frames.colnames:
    if hasattr(jura_frames[c], 'mask'):
        print(c)

In [ ]:
jura_exposures['MJD'].min()

In [ ]:
jura_exposures_bad_rows = list()
bad_columns = list()
for c in jura_exposures.colnames:
    if hasattr(jura_exposures[c], 'mask'):
        print(c)
        bad_columns.append(c)
        jura_exposures_bad_rows.append(np.where(jura_exposures[c].mask)[0])
    elif jura_exposures[c].dtype.kind == 'f' and not np.isfinite(jura_exposures[c]).all():
        print(c)
        bad_columns.append(c)
        jura_exposures_bad_rows.append(np.where(~np.isfinite(jura_exposures[c]))[0])
    else:
        pass
jura_exposures_bad_rows = np.unique(np.hstack(jura_exposures_bad_rows))
jura_exposures[jura_exposures_bad_rows]

### All exposures for the tiles that contain bad exposures

In [ ]:
jura_exposures_bad_tiles = np.in1d(jura_exposures['TILEID'], jura_exposures[jura_exposures_bad_rows]['TILEID'])
jura_exposures[jura_exposures_bad_tiles]

In [ ]:
compute_efftime(jura_exposures[jura_exposures_bad_tiles])

In [ ]:
jura_exposures_test = jura_exposures[jura_exposures_bad_tiles].copy()
for c in bad_columns:
    bad_rows = ~np.isfinite(jura_exposures_test[c])
    print("Replacing {0:d} bad values in column {1}.".format(np.sum(bad_rows), c))
    jura_exposures_test[c][bad_rows] = 0
jura_exposures_test

In [ ]:
a, b, c = compute_efftime(jura_exposures_test)
assert np.isfinite(a).all()
assert np.isfinite(b).all()
assert np.isfinite(c).all()

In [ ]:
jura_tiles_bad_rows = np.in1d(jura_tiles['TILEID'], jura_exposures[jura_exposures_bad_rows]['TILEID'])
jura_tiles[jura_tiles_bad_rows]

### What about in iron?

In [ ]:
iron_tiles[iron_tiles_bad_rows]

In [ ]:
iron_exposures[(iron_exposures['TILEID'] == 42262) | (iron_exposures['TILEID'] == 40203)]

In [ ]:
fuji_exposures_bad_rows = np.in1d(fuji_exposures['EXPID'], jura_exposures[jura_exposures_bad_rows]['EXPID'])
fuji_exposures[fuji_exposures_bad_rows]

In [ ]:
fuji_exposures[(fuji_exposures['EXPID'] > 74300) & (fuji_exposures['EXPID'] < 74500)]

### What about the GFA file?



In [ ]:
gfa_sv1_file = os.path.join(os.environ['DESI_ROOT'], 'survey', 'GFA', 'offline_matched_coadd_ccds_SV1-thru_20210928.fits')
gfa_sv3_file = os.path.join(os.environ['DESI_ROOT'], 'survey', 'GFA.KPNO', 'offline_matched_coadd_ccds_SV3-thru_20220201.fits') # fuji
# gfa_sv3_file = os.path.join(os.environ['DESI_ROOT'], 'survey', 'GFA.KPNO', 'offline_matched_coadd_ccds_SV3-thru_20230207.fits') # iron 
# gfa_sv3_file = os.path.join(os.environ['DESI_ROOT'], 'survey', 'GFA', 'offline_matched_coadd_ccds_SV3-thru_20240409.fits') # jura
gfa_sv1_table = read_table(gfa_sv1_file, ext=2)
gfa_sv3_table = read_table(gfa_sv3_file, ext=2)

In [ ]:
gfa_sv1_table_bad_rows = np.in1d(gfa_sv1_table['EXPID'], jura_exposures[jura_exposures_bad_tiles]['EXPID'])
gfa_sv1_table[gfa_sv1_table_bad_rows]

In [ ]:
gfa_sv3_table_bad_rows = np.in1d(gfa_sv3_table['EXPID'], jura_exposures[jura_exposures_bad_tiles]['EXPID'])
gfa_sv3_table[gfa_sv3_table_bad_rows]

## Find jura exposures not in daily, daily exposures not in jura

In [3]:
assert (np.unique(jura_exposures['EXPID']) == sorted(jura_exposures['EXPID'])).all()
assert (np.unique(daily_exposures['EXPID']) == sorted(daily_exposures['EXPID'])).all()
assert (np.unique(jura_frames['EXPID']) == sorted(jura_exposures['EXPID'])).all()
assert (np.unique(daily_frames['EXPID']) == sorted(daily_exposures['EXPID'])).all()

In [4]:
first_jura_exposure, last_jura_exposure = jura_exposures['EXPID'].min(), jura_exposures['EXPID'].max()
first_jura_exposure, last_jura_exposure

(67710, 235203)

In [5]:
first_jura_night = jura_exposures['NIGHT'][jura_exposures['EXPID'] == first_jura_exposure].min()
last_jura_night = jura_exposures['NIGHT'][jura_exposures['EXPID'] == last_jura_exposure].max()
first_jura_night, last_jura_night

(20201214, 20240409)

In [6]:
jura_tiles['LASTNIGHT'].min(), jura_tiles['LASTNIGHT'].max()

(20201216, 20240409)

In [7]:
jura_expid_set = frozenset(jura_exposures['EXPID'].tolist())
daily_expid_set = frozenset(daily_exposures['EXPID'].tolist())

In [8]:
jura_not_in_daily = jura_expid_set - daily_expid_set
jura_not_in_daily

frozenset({80478, 80681, 80688, 80691, 82603, 82622, 82625, 221977})

In [9]:
daily_not_in_jura = daily_expid_set - jura_expid_set
# daily_not_in_jura

## Patch frames

We don't necessarily want to change the values in the `FRAMES` table, just make sure it is consistent with the `EXPOSURES` table.

In [10]:
daily_frames_join = Table()
daily_frames_join['FRAMEID'] = np.array([100*row['EXPID'] + cameraid(row['CAMERA']) for row in daily_frames])
daily_frames_join['DAILY_INDEX'] = np.arange(len(daily_frames))

In [11]:
jura_frames_join = Table()
jura_frames_join['FRAMEID'] = np.array([100*row['EXPID'] + cameraid(row['CAMERA']) for row in jura_frames])
jura_frames_join['JURA_INDEX'] = np.arange(len(jura_frames))

In [12]:
j = join(daily_frames_join, jura_frames_join, join_type='outer', keys='FRAMEID')
# j

In [13]:
daily_frames_index = j[(~j['JURA_INDEX'].mask) & (~j['DAILY_INDEX'].mask)]['DAILY_INDEX']
jura_frames_index = j[(~j['JURA_INDEX'].mask) & (~j['DAILY_INDEX'].mask)]['JURA_INDEX']

In [14]:
daily_frames_patched = daily_frames.copy()
for column in daily_frames_patched.colnames:
    if hasattr(daily_frames_patched[column], 'mask') and not column.startswith('TSNR2_'):
        print(f"Patching {column}...")
        daily_frames_patched[column][daily_frames_index] = jura_frames[column][jura_frames_index]
        daily_frames_patched[column].mask[daily_frames_index] = False
# daily_frames_patched

Patching SEEING_ETC...
Patching EBV...


## Patch exposures

We want to only *patch* and rows that:

* Appear in `jura`.
* Have `NIGHT >= first_jura_night`.
* Have `EFFTIME_SPEC > 0`.

We want to only *load* rows that:

* Have `NIGHT >= first_jura_night`.
* Have `EFFTIME_SPEC > 0`.

which is slightly different. However we don't want to *remove* rows that *don't* satisfy these criteria.

In [47]:
daily_exposures_join = Table()
daily_exposures_join['EXPID'] = daily_exposures['EXPID']
# daily_exposures_join['NIGHT'] = daily_exposures['NIGHT']
# daily_exposures_join['EFFTIME_SPEC'] = daily_exposures['EFFTIME_SPEC']
daily_exposures_join['DAILY_INDEX'] = np.arange(len(daily_exposures))

In [48]:
jura_exposures_join = Table()
jura_exposures_join['EXPID'] = jura_exposures['EXPID']
jura_exposures_join['JURA_INDEX'] = np.arange(len(jura_exposures))

In [49]:
j = join(daily_exposures_join, jura_exposures_join, join_type='outer', keys='EXPID')
j

EXPID,DAILY_INDEX,JURA_INDEX
int32,int64,int64
50986,0,--
50988,1,--
50991,2,--
50995,3,--
51001,4,--
51002,5,--
51028,6,--
51029,7,--
51030,8,--


In [50]:
daily_exposures_index = j[(~j['JURA_INDEX'].mask) & (~j['DAILY_INDEX'].mask)]['DAILY_INDEX']
jura_exposures_index = j[(~j['JURA_INDEX'].mask) & (~j['DAILY_INDEX'].mask)]['JURA_INDEX']

In [51]:
daily_exposures_patched = daily_exposures.copy()
can_patch = ('NIGHT', 'EXPID', 'TILEID', 'TILERA', 'TILEDEC', 'MJD', 'SURVEY', 'PROGRAM', 'FAPRGRM', 'FAFLAVOR', 'EXPTIME', 'GOALTIME', 'GOALTYPE', 'MINTFRAC', 'AIRMASS', 'EBV', 'SEEING_ETC', 'EFFTIME_ETC',
             'TRANSPARENCY_GFA', 'SEEING_GFA', 'FIBER_FRACFLUX_GFA', 'FIBER_FRACFLUX_ELG_GFA', 'FIBER_FRACFLUX_BGS_GFA', 'FIBERFAC_GFA', 'FIBERFAC_ELG_GFA', 'FIBERFAC_BGS_GFA', 'AIRMASS_GFA', 'SKY_MAG_AB_GFA',
             'EFFTIME_GFA', 'EFFTIME_DARK_GFA', 'EFFTIME_BRIGHT_GFA', 'EFFTIME_BACKUP_GFA')
for column in ['TILERA', 'TILEDEC', 'MJD', 'SURVEY'] + [c for c in daily_exposures_patched.colnames if hasattr(daily_exposures_patched[c], 'mask') and c in can_patch]:
    print(f"Patching {column}...")
    if hasattr(jura_exposures[column], 'mask'):
        if np.any(jura_exposures[column].mask[jura_exposures_index]):
            jura_exposures[column][jura_exposures[column].mask] = 0
            jura_exposures[column].mask[jura_exposures[column].mask] = False
    daily_exposures_patched[column][daily_exposures_index] = jura_exposures[column][jura_exposures_index]
    if hasattr(daily_exposures_patched[column], 'mask'):
        daily_exposures_patched[column].mask[daily_exposures_index] = False

known_bad_exposures = np.in1d(daily_exposures_patched['EXPID'], np.array([74307, 79769, 83420, 110852, 190752]))
daily_exposures_patched[known_bad_exposures]

Patching TILERA...
Patching TILEDEC...
Patching MJD...
Patching SURVEY...
Patching EBV...
Patching SEEING_ETC...
Patching TRANSPARENCY_GFA...
Patching SEEING_GFA...
Patching FIBER_FRACFLUX_GFA...
Patching FIBER_FRACFLUX_ELG_GFA...
Patching FIBER_FRACFLUX_BGS_GFA...
Patching FIBERFAC_GFA...
Patching FIBERFAC_ELG_GFA...
Patching FIBERFAC_BGS_GFA...
Patching EFFTIME_GFA...
Patching EFFTIME_DARK_GFA...
Patching EFFTIME_BRIGHT_GFA...
Patching EFFTIME_BACKUP_GFA...


NIGHT,EXPID,TILEID,TILERA,TILEDEC,MJD,SURVEY,PROGRAM,FAPRGRM,FAFLAVOR,EXPTIME,EFFTIME_SPEC,GOALTIME,GOALTYPE,MINTFRAC,AIRMASS,EBV,SEEING_ETC,EFFTIME_ETC,TSNR2_ELG,TSNR2_QSO,TSNR2_LRG,TSNR2_LYA,TSNR2_BGS,TSNR2_GPBDARK,TSNR2_GPBBRIGHT,TSNR2_GPBBACKUP,LRG_EFFTIME_DARK,ELG_EFFTIME_DARK,BGS_EFFTIME_BRIGHT,LYA_EFFTIME_DARK,GPB_EFFTIME_DARK,GPB_EFFTIME_BRIGHT,GPB_EFFTIME_BACKUP,TRANSPARENCY_GFA,SEEING_GFA,FIBER_FRACFLUX_GFA,FIBER_FRACFLUX_ELG_GFA,FIBER_FRACFLUX_BGS_GFA,FIBERFAC_GFA,FIBERFAC_ELG_GFA,FIBERFAC_BGS_GFA,AIRMASS_GFA,SKY_MAG_AB_GFA,SKY_MAG_G_SPEC,SKY_MAG_R_SPEC,SKY_MAG_Z_SPEC,EFFTIME_GFA,EFFTIME_DARK_GFA,EFFTIME_BRIGHT_GFA,EFFTIME_BACKUP_GFA
int32,int32,int32,float64,float64,float64,bytes7,bytes6,bytes19,bytes19,float64,float64,float64,bytes7,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
20210202,74307,80679,111.0,41.5,59248.23876322,sv1,dark,elgqso,sv1elgqso,566.04931640625,1.4423755109310155,4000.0,dark,0.9,1.014901,0.05597689002752304,0.0,0.0,0.1677180826663971,0.04438820853829384,0.11977556347846985,0.1674174666404724,11.726827621459961,0.0,0.0,0.0,1.455345647106182,1.4423755109310155,1.6417558670043948,1.9152153084460037,0.0,0.0,0.0,0.059126717922222294,0.9789517068362219,0.6320627694913908,0.44970044302469636,0.20504407421980816,0.0,0.0,0.0,1.0143504042229632,21.72197341353745,21.95553631921876,21.212220164279294,20.113056586954166,0.0,0.0,0.0,0.0
20210308,79769,80731,192.86,27.13,59282.39481033,sv1,other,ssv,sv1ssv,300.06988525390625,4.908081118628617e-05,1000.0,unknown,0.9,1.006417,0.010517628863453865,0.0,0.0,5.707071068172809e-06,1.5589646409353008e-06,3.6969945540477056e-06,6.8609160734922625e-06,0.0003828057087957859,0.0,0.0,0.0,4.4920723187211217e-05,4.908081118628617e-05,5.3592799231410035e-05,7.848721974830708e-05,0.0,0.0,0.0,0.0,1.8761397170576326,0.2613052900585784,0.21108856454109742,0.10411062155477979,0.0,0.0,0.0,1.0072026342866551,21.440068174240203,22.274960404546178,21.280958712336684,19.68924689265133,0.0,0.0,0.0,0.0
20210404,83420,80697,145.0,32.375,59309.20018021,sv1,dark,elg,sv1elg,80.4530029296875,4.205021150482936e-05,4000.0,dark,0.9,1.0092869997024536,0.018713951110839844,0.0,0.0,4.889559477305738e-06,9.873140243144007e-07,4.668986548495013e-06,6.542905478545882e-07,0.00034244905691593885,0.0,0.0,0.0,5.67310146778893e-05,4.205021150482936e-05,4.7942867968231444e-05,7.484925549098573e-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0007895831162312318,0.0006804336788210475,0.00030002771194605096,1.0096259147044822,20.890481710495784,21.92729853866057,20.87627884543552,19.709699199758923,1.0075802546981036e-05,1.0075802546981036e-05,0.0,0.0
20211125,110852,42262,5.991,4.792,59544.11509409,main,backup,backup,mainbackup,601.6654663085938,17.011510848999023,60.0,backup,0.85,1.143367052078247,0.023625371977686882,3.7747349739074707,9.584416389465332,2.1469082832336426,0.3637787997722626,1.6068999767303467,0.8854541587585117,150.91136169433594,197.9574432373047,42.43981170654297,240.1890869140625,19.524808883666992,18.463411331176758,21.12759017944336,10.449163963405304,16.594938278198242,22.2078800201416,17.011510848999023,0.8780836223539111,3.20778365614237,0.11836103479224255,0.10716889904754712,0.057902263978654445,0.0,0.0,0.0,1.1394144832564999,20.885443339512,21.82595186409396,20.871110333724605,19.155375680908563,0.0,0.0,0.0,0.0
20230815,190752,40203,320.573,26.421,60172.30414529,main,backup,backup,mainbackup,603.0149536132812,0.6952639818191528,60.0,backup,0.85,1.0050499439239502,0.11382203549146652,1.1314209699630737,0.5669659972190857,0.04952569305896759,0.01299299392849207,0.03662736713886261,0.048888119613252456,3.586458444595337,5.241128921508789,1.040251612663269,9.816577911376953,0.44504472613334656,0.4259209930896759,0.5021041631698608,0.5769242514120

### After patching are there still missing data?

In [52]:
assert not (daily_exposures_patched['TILERA'] == daily_exposures['TILERA']).all()
assert not (daily_exposures_patched['TILEDEC'] == daily_exposures['TILEDEC']).all()
assert not (daily_exposures_patched['MJD'] == daily_exposures['MJD']).all()
assert not (daily_exposures_patched['SURVEY'] == daily_exposures['SURVEY']).all()
assert (daily_exposures_patched['PROGRAM'] == daily_exposures['PROGRAM']).all()
assert (daily_exposures_patched['FAPRGRM'] == daily_exposures['FAPRGRM']).all()
assert (daily_exposures_patched['FAFLAVOR'] == daily_exposures['FAFLAVOR']).all()

In [53]:
missing_mjd = np.where((daily_exposures_patched['NIGHT'] >= first_jura_night) & (daily_exposures_patched['EFFTIME_SPEC'] > 0) & (daily_exposures_patched['MJD'] < 50000))[0]
for row in daily_exposures_patched[missing_mjd]:
    raw_data_file = os.path.join(os.environ['DESI_SPECTRO_DATA'], "{0:08d}".format(row['NIGHT']), "{0:08d}".format(row['EXPID']), "desi-{0:08d}.fits.fz".format(row['EXPID']))
    with fits.open(raw_data_file, mode='readonly') as hdulist:
        mjd_obs = hdulist['SPEC'].header['MJD-OBS']
    print("INFO: tile {0:d} exposure {1:d} has MJD-OBS = {2:f} in {3}!".format(row['TILEID'], row['EXPID'], mjd_obs, raw_data_file))
    w = np.where(daily_exposures_patched['EXPID'] == row['EXPID'])[0]
    assert len(w) == 1
    daily_exposures_patched['MJD'][w] = mjd_obs
    print(daily_exposures_patched[w][['NIGHT', 'EXPID', 'TILEID', 'MJD']])

INFO: tile 80618 exposure 69082 has MJD-OBS = 59204.557878 in /global/cfs/cdirs/desi/spectro/data/20201220/00069082/desi-00069082.fits.fz!
 NIGHT   EXPID TILEID      MJD      
-------- ----- ------ --------------
20201220 69082  80618 59204.55787815
INFO: tile 80618 exposure 69083 has MJD-OBS = 59204.563137 in /global/cfs/cdirs/desi/spectro/data/20201220/00069083/desi-00069083.fits.fz!
 NIGHT   EXPID TILEID      MJD      
-------- ----- ------ --------------
20201220 69083  80618 59204.56313729
INFO: tile 80618 exposure 69084 has MJD-OBS = 59204.568494 in /global/cfs/cdirs/desi/spectro/data/20201220/00069084/desi-00069084.fits.fz!
 NIGHT   EXPID TILEID      MJD      
-------- ----- ------ --------------
20201220 69084  80618 59204.56849373
INFO: tile 80618 exposure 69085 has MJD-OBS = 59204.573527 in /global/cfs/cdirs/desi/spectro/data/20201220/00069085/desi-00069085.fits.fz!
 NIGHT   EXPID TILEID      MJD      
-------- ----- ------ --------------
20201220 69085  80618 59204.57352664


In [54]:
still_missing_mjd = np.where((daily_exposures_patched['NIGHT'] >= first_jura_night) & (daily_exposures_patched['EFFTIME_SPEC'] > 0) & (daily_exposures_patched['MJD'] < 50000))[0]
daily_exposures_patched[still_missing_mjd]

NIGHT,EXPID,TILEID,TILERA,TILEDEC,MJD,SURVEY,PROGRAM,FAPRGRM,FAFLAVOR,EXPTIME,EFFTIME_SPEC,GOALTIME,GOALTYPE,MINTFRAC,AIRMASS,EBV,SEEING_ETC,EFFTIME_ETC,TSNR2_ELG,TSNR2_QSO,TSNR2_LRG,TSNR2_LYA,TSNR2_BGS,TSNR2_GPBDARK,TSNR2_GPBBRIGHT,TSNR2_GPBBACKUP,LRG_EFFTIME_DARK,ELG_EFFTIME_DARK,BGS_EFFTIME_BRIGHT,LYA_EFFTIME_DARK,GPB_EFFTIME_DARK,GPB_EFFTIME_BRIGHT,GPB_EFFTIME_BACKUP,TRANSPARENCY_GFA,SEEING_GFA,FIBER_FRACFLUX_GFA,FIBER_FRACFLUX_ELG_GFA,FIBER_FRACFLUX_BGS_GFA,FIBERFAC_GFA,FIBERFAC_ELG_GFA,FIBERFAC_BGS_GFA,AIRMASS_GFA,SKY_MAG_AB_GFA,SKY_MAG_G_SPEC,SKY_MAG_R_SPEC,SKY_MAG_Z_SPEC,EFFTIME_GFA,EFFTIME_DARK_GFA,EFFTIME_BRIGHT_GFA,EFFTIME_BACKUP_GFA
int32,int32,int32,float64,float64,float64,bytes7,bytes6,bytes19,bytes19,float64,float64,float64,bytes7,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64


In [59]:
possible_expid_still_patchable = list()
for c in daily_exposures_patched.colnames:
    if hasattr(daily_exposures_patched[c], 'mask'):
        if daily_exposures_patched[c].mask.any():
            n_masked = daily_exposures_patched[c].mask.sum()
            not_in_jura = [e in daily_not_in_jura for e in daily_exposures_patched['EXPID'][daily_exposures_patched[c].mask]]
            if all(not_in_jura):
                print(f"Column {c} still has {n_masked:d} masked values, but the exposures are not in jura and cannot be patched.")
            else:
                if c in can_patch:
                    for e in daily_exposures_patched['EXPID'][daily_exposures_patched[c].mask]:
                        if e not in daily_not_in_jura:
                            possible_expid_still_patchable.append(int(e))
                    print(f"Column {c} still has {n_masked:d} masked values, and some exposures can still be patched.")
                else:
                    print(f"Column {c} still has {n_masked:d} masked values, and some exposures could still be patched, but they do not meet the patchable column guidelines.")

possible_expid_still_patchable = np.unique(np.array(possible_expid_still_patchable))
daily_exposures_patched[np.in1d(daily_exposures_patched['EXPID'], possible_expid_still_patchable)]

Column EFFTIME_SPEC still has 1 masked values, but the exposures are not in jura and cannot be patched.
Column EBV still has 12 masked values, but the exposures are not in jura and cannot be patched.
Column SEEING_ETC still has 1 masked values, but the exposures are not in jura and cannot be patched.
Column TSNR2_ELG still has 1 masked values, but the exposures are not in jura and cannot be patched.
Column TSNR2_QSO still has 1 masked values, but the exposures are not in jura and cannot be patched.
Column TSNR2_LRG still has 1 masked values, but the exposures are not in jura and cannot be patched.
Column TSNR2_LYA still has 1 masked values, but the exposures are not in jura and cannot be patched.
Column TSNR2_BGS still has 1 masked values, but the exposures are not in jura and cannot be patched.
Column TSNR2_GPBDARK still has 1 masked values, but the exposures are not in jura and cannot be patched.
Column TSNR2_GPBBRIGHT still has 1 masked values, but the exposures are not in jura and 

NIGHT,EXPID,TILEID,TILERA,TILEDEC,MJD,SURVEY,PROGRAM,FAPRGRM,FAFLAVOR,EXPTIME,EFFTIME_SPEC,GOALTIME,GOALTYPE,MINTFRAC,AIRMASS,EBV,SEEING_ETC,EFFTIME_ETC,TSNR2_ELG,TSNR2_QSO,TSNR2_LRG,TSNR2_LYA,TSNR2_BGS,TSNR2_GPBDARK,TSNR2_GPBBRIGHT,TSNR2_GPBBACKUP,LRG_EFFTIME_DARK,ELG_EFFTIME_DARK,BGS_EFFTIME_BRIGHT,LYA_EFFTIME_DARK,GPB_EFFTIME_DARK,GPB_EFFTIME_BRIGHT,GPB_EFFTIME_BACKUP,TRANSPARENCY_GFA,SEEING_GFA,FIBER_FRACFLUX_GFA,FIBER_FRACFLUX_ELG_GFA,FIBER_FRACFLUX_BGS_GFA,FIBERFAC_GFA,FIBERFAC_ELG_GFA,FIBERFAC_BGS_GFA,AIRMASS_GFA,SKY_MAG_AB_GFA,SKY_MAG_G_SPEC,SKY_MAG_R_SPEC,SKY_MAG_Z_SPEC,EFFTIME_GFA,EFFTIME_DARK_GFA,EFFTIME_BRIGHT_GFA,EFFTIME_BACKUP_GFA
int32,int32,int32,float64,float64,float64,bytes7,bytes6,bytes19,bytes19,float64,float64,float64,bytes7,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64


### Fill remaining masked values with zero.

In [60]:
for c in daily_exposures_patched.colnames:
    if hasattr(daily_exposures_patched[c], 'mask'):
        if daily_exposures_patched[c].mask.any():
            daily_exposures_patched[c][daily_exposures_patched[c].mask] = 0
            daily_exposures_patched[c].mask[daily_exposures_patched[c].mask] = False

## Patch tiles

Similar to the discussion above, we want `LASTNIGHT >= first_jura_night`.

In [63]:
assert (np.unique(jura_exposures['TILEID']) == sorted(jura_tiles['TILEID'])).all()
assert (np.unique(daily_exposures['TILEID']) == sorted(daily_tiles['TILEID'])).all()

In [64]:
daily_tiles_join = Table()
daily_tiles_join['TILEID'] = daily_tiles['TILEID']
# daily_tiles_join['LASTNIGHT'] = daily_tiles['LASTNIGHT']
# daily_tiles_join['EFFTIME_SPEC'] = daily_tiles['EFFTIME_SPEC']
daily_tiles_join['DAILY_INDEX'] = np.arange(len(daily_tiles))

In [65]:
jura_tiles_join = Table()
jura_tiles_join['TILEID'] = jura_tiles['TILEID']
jura_tiles_join['JURA_INDEX'] = np.arange(len(jura_tiles))

In [66]:
j = join(daily_tiles_join, jura_tiles_join, join_type='outer', keys='TILEID')
j

TILEID,DAILY_INDEX,JURA_INDEX
int64,int64,int64
1,432,202
2,436,230
3,471,267
4,502,292
5,533,321
6,611,400
7,736,476
8,748,486
9,779,513


In [67]:
# Apparently every tile in daily also appears in jura, so j['DAILY_INDEX'] doesn't need to be masked.
daily_tiles_index = j[(~j['JURA_INDEX'].mask)]['DAILY_INDEX']
jura_tiles_index = j[(~j['JURA_INDEX'].mask)]['JURA_INDEX']

In [68]:
daily_tiles_patched = daily_tiles.copy()
# for column in jura_tiles.colnames:
# for column in ('TILEID', 'SURVEY', 'PROGRAM', 'FAPRGRM', 'FAFLAVOR', 'TILERA', 'TILEDEC', 'EFFTIME_ETC', 'EFFTIME_GFA', 'GOALTIME', 'GOALTYPE', 'MINTFRAC'):
for column in ('PROGRAM', ):
    daily_tiles_patched[column][daily_tiles_index] = jura_tiles[column][jura_tiles_index]
    daily_tiles_patched[column].mask[daily_tiles_index] = False
daily_tiles_patched

TILEID,SURVEY,PROGRAM,FAPRGRM,FAFLAVOR,NEXP,EXPTIME,TILERA,TILEDEC,EFFTIME_ETC,EFFTIME_SPEC,EFFTIME_GFA,GOALTIME,OBSSTATUS,LRG_EFFTIME_DARK,ELG_EFFTIME_DARK,BGS_EFFTIME_BRIGHT,LYA_EFFTIME_DARK,GOALTYPE,MINTFRAC,LASTNIGHT
int64,str7,str6,str16,str19,int64,float64,float64,float64,float64,float64,float64,float64,str8,float64,float64,float64,float64,str7,float64,int64
70004,unknown,--,unknown,unknown,4,3600.0,116.0,20.7,0.0,3619.8,0.0,1000.0,obsend,3470.8,3619.8,3784.0,3056.6,unknown,0.9,20200219
70508,unknown,--,unknown,unknown,6,1800.0,133.4125,11.6818,0.0,108.5,0.0,1000.0,obsstart,93.6,108.5,95.7,50.6,unknown,0.9,20200225
70506,unknown,--,unknown,unknown,3,900.0,133.4125,11.6818,0.0,61.6,0.0,1000.0,obsstart,53.6,61.6,54.9,33.7,unknown,0.9,20200225
70512,unknown,--,unknown,unknown,7,3150.0,132.85,12.32,0.0,389.8,0.0,1000.0,obsstart,351.9,389.8,364.7,227.2,unknown,0.9,20200226
70514,unknown,--,unknown,unknown,16,1680.0,132.85,12.32,0.0,118.3,0.0,1000.0,obsstart,112.0,118.3,120.2,74.4,unknown,0.9,20200227
70502,unknown,--,unknown,unknown,41,8415.0,180.0,-0.5,0.0,358.2,0.0,1000.0,obsstart,317.9,358.2,325.0,260.2,unknown,0.9,20200227
70513,unknown,--,unknown,unknown,13,2220.0,133.42,11.65,0.0,3.7,0.0,1000.0,obsstart,3.4,3.7,3.5,2.5,unknown,0.9,20200229
70500,unknown,--,unknown,unknown,14,7000.0,119.0,50.0,0.0,1389.0,0.0,1000.0,obsend,1145.4,1389.0,1160.5,770.4,unknown,0.9,20200303
70005,unknown,--,unknown,unknown,17,13500.0,158.0,25.0,0.0,8463.1,0.0,1000.0,obsend,8088.5,8463.1,8858.6,9174.7,unknown,0.9,20200303


### After patching are there still missing data?

Also does `GOALTYPE` actually need to be patched? Should we patch `EFFTIME_SPEC`?

In [69]:
missing_program = np.where((daily_tiles_patched['LASTNIGHT'] >= first_jura_night) & (daily_tiles_patched['EFFTIME_SPEC'] > 0) & (daily_tiles_patched['PROGRAM'].mask))[0]
daily_tiles_patched[missing_program]

TILEID,SURVEY,PROGRAM,FAPRGRM,FAFLAVOR,NEXP,EXPTIME,TILERA,TILEDEC,EFFTIME_ETC,EFFTIME_SPEC,EFFTIME_GFA,GOALTIME,OBSSTATUS,LRG_EFFTIME_DARK,ELG_EFFTIME_DARK,BGS_EFFTIME_BRIGHT,LYA_EFFTIME_DARK,GOALTYPE,MINTFRAC,LASTNIGHT
int64,str7,str6,str16,str19,int64,float64,float64,float64,float64,float64,float64,float64,str8,float64,float64,float64,float64,str7,float64,int64
80713,sv1,--,m31,sv1m31,3,2700.2,10.17,41.38,0.0,758.6,771.8,1000.0,obsstart,727.8,758.6,789.8,645.0,unknown,0.9,20210110
80715,sv1,--,m31,sv1m31,3,2700.1,10.17,41.38,0.0,1906.8,1827.2,1000.0,obsend,1914.5,1906.8,2150.1,2216.6,unknown,0.9,20210115
80769,unknown,--,dithfocus,dithfocus,7,920.3,111.64,30.0,0.0,1.2,0.0,1000.0,obsstart,1.1,1.2,1.2,0.9,unknown,0.9,20210219
80816,cmx,--,dithprec,dithprec,3,540.2,99.0,66.0,0.0,14.1,45.2,1000.0,obsstart,11.4,14.1,11.6,6.8,unknown,0.9,20210223
80916,unknown,--,dithfocus,dithfocus,4,740.2,160.0,50.0,0.0,63.6,68.8,1000.0,obsstart,59.7,63.6,66.5,67.5,unknown,0.9,20210422
82026,unknown,--,dithprec,dithprec,1,200.1,251.2,50.4,0.0,73.1,59.4,1000.0,obsstart,67.5,73.1,72.7,64.4,unknown,0.9,20210423
82065,unknown,--,dithfocus,dithfocus,1,200.1,220.2,47.8,0.0,24.8,27.9,1000.0,obsstart,22.4,24.8,24.5,25.4,unknown,0.9,20210423
82052,unknown,--,dithfocus,dithfocus,1,200.1,190.0,49.5,0.0,30.1,17.3,1000.0,obsstart,25.4,30.1,26.1,14.9,unknown,0.9,20210423
81096,sv2,--,dark,sv2dark,1,1220.1,180.0,45.0,0.0,1193.2,973.8,1000.0,obsend,1111.7,1193.2,1234.4,1459.6,dark,0.9,20210429


## Write out the patched files

In [70]:
daily_tiles_patched.write(os.path.join(os.environ['DESI_ROOT'], 'users', os.environ['USER'], 'tiles-daily-patched-with-jura.csv'), format='ascii.csv', overwrite=True)

In [71]:
daily_exposures_patched.write(os.path.join(os.environ['DESI_ROOT'], 'users', os.environ['USER'], 'exposures-daily-patched-with-jura.csv'), format='ascii.csv', overwrite=True)

In [72]:
daily_exposures_fits = fits.HDUList([fits.PrimaryHDU(), fits.table_to_hdu(daily_exposures_patched), fits.table_to_hdu(daily_frames_patched)])

In [73]:
daily_exposures_fits.info()

Filename: (No file associated with this HDUList)
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1  EXPOSURES     1 BinTableHDU    111   23895R x 51C   ['J', 'J', 'J', 'D', 'D', 'D', '7A', '6A', '19A', '19A', 'D', 'D', 'D', '7A', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D']   
  2  FRAMES        1 BinTableHDU     63   703435R x 27C   ['J', 'J', 'J', 'E', 'D', 'D', '2A', 'D', 'D', 'D', 'D', '7A', '7A', '19A', 'D', 'D', 'D', 'D', '19A', 'E', 'E', 'D', 'D', 'D', 'D', 'D', 'D']   


In [74]:
daily_exposures_fits.writeto(os.path.join(os.environ['DESI_ROOT'], 'users', os.environ['USER'], 'exposures-daily-patched-with-jura.fits'), overwrite=True)